<a href="https://colab.research.google.com/github/llHyun/korea-trip-tsp/blob/main/korea_trip_tsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 필수 라이브러리 설치 (Colab에서만 1회 실행)
!pip install osmnx networkx

# 🔧 라이브러리 불러오기
import osmnx as ox
import networkx as nx
import pandas as pd
import itertools
from math import ceil

# ✅ 사용자 입력
여행일수 = 3
출발지 = "솔샘역"
마지막목적지 = "성수역"
목적지들 = [
    "북촌한옥마을",
    "인사동",
    "경복궁",
    "망원시장",
    "남산서울타워",
    "서울숲",
    "서울특별시 성동구 동일로 143"
]

숙소정보 = {
    "Day1": {"숙소명": "솔샘역", "짐놓기": True, "휴식": False},
    "Day2": {"숙소명": "솔샘역", "짐놓기": False, "휴식": True},
    "Day3": {"숙소명": "성수역", "짐놓기": False, "휴식": False},
}

일정강도 = {
    "Day1": "빡빡",   # 3
    "Day2": "중간",   # 2
    "Day3": "휴식"    # 0
}

# ✅ 가중치 계산
강도값 = {"빡빡": 3, "중간": 2, "널널": 1, "휴식": 0}
가중치목록 = {day: 강도값[일정강도[day]] for day in 일정강도}
총가중치 = sum(가중치목록.values())

# ✅ 목적지 균등 분배
목적지_리스트 = [p for p in 목적지들 if p != 마지막목적지]
배정수 = {
    day: round(len(목적지_리스트) * (가중치 / 총가중치))
    for day, 가중치 in 가중치목록.items()
}

# 오차 보정
남은 = len(목적지_리스트) - sum(배정수.values())
if 남은 != 0:
    sorted_days = sorted(배정수.items(), key=lambda x: -x[1])
    for i in range(abs(남은)):
        d = sorted_days[i % len(sorted_days)][0]
        배정수[d] += 1 if 남은 > 0 else -1

# 실제 분배
숙소_배정 = {day: [] for day in 일정강도}
idx = 0
for day in 일정강도:
    숙소_배정[day] = 목적지_리스트[idx:idx + 배정수[day]]
    idx += 배정수[day]
숙소_배정[list(일정강도)[-1]].append(마지막목적지)

# ✅ 지도 불러오기
G = ox.graph_from_place("Seoul, South Korea", network_type='drive')

# ✅ 지오코딩
전체지점 = 목적지들 + [출발지, 마지막목적지] + [숙소정보[d]["숙소명"] for d in 숙소정보]
지오코딩 = {}
for p in 전체지점:
    try:
        지오코딩[p] = ox.geocode(p + ", 서울")
    except:
        print(f"지오코딩 실패: {p}")

노드맵 = {k: ox.distance.nearest_nodes(G, lon, lat) for k, (lat, lon) in 지오코딩.items()}

# ✅ 거리 계산 함수
def 경로거리(G, 순서):
    거리 = 0
    for i in range(len(순서)-1):
        try:
            path = nx.shortest_path_length(G, 노드맵[순서[i]], 노드맵[순서[i+1]], weight='length')
            거리 += path
        except:
            거리 += 1e9
    return 거리 / 1000  # km

# ✅ 최적 경로 계산
결과 = []
for day, 장소들 in 숙소_배정.items():
    if 강도값[일정강도[day]] == 0:
        결과.append([day, 1, 숙소정보[day]["숙소명"], 0])
        continue

    출발 = 숙소정보[day]["숙소명"] if 숙소정보[day]["짐놓기"] else 출발지
    경유지 = 장소들.copy()
    if 숙소정보[day]["휴식"]:
        중간숙소 = 숙소정보[day]["숙소명"]
        경유지.insert(len(경유지)//2, 중간숙소)
    순열 = list(itertools.permutations(경유지))
    최적 = min(순열, key=lambda x: 경로거리(G, [출발] + list(x) + [숙소정보[day]["숙소명"]]))
    최적경로 = [출발] + list(최적) + [숙소정보[day]["숙소명"]]

    거리 = 경로거리(G, 최적경로)
    for i, p in enumerate(최적경로):
        결과.append([day, i+1, p, 거리 if i == 0 else None])

# ✅ 결과 출력
df = pd.DataFrame(결과, columns=["일자", "순번", "장소", "총 거리 (km)"])
df


,일자,순번,장소,총 거리 (km)
0,Day1,1,솔샘역,32.337368
1,Day1,2,북촌한옥마을,NaN
2,Day1,3,인사동,NaN
3,Day1,4,경복궁,NaN
4,Day1,5,망원시장,NaN
5,Day1,6,솔샘역,NaN
6,Day2,1,솔샘역,32.664689
7,Day2,2,남산서울타워,NaN
8,Day2,3,서울숲,NaN
9,Day2,4,서울특별시 성동구 동일로 143,NaN
